In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [2]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [3]:
import neural_ode.ODESolvers

In [4]:
tf.keras.backend.set_floatx('float64')

In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=(2,)),
        layers.Dense(2, name="layer1"),
    ]
)


In [6]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2, 
                                       solver=neural_ode.ODESolvers.BackwardEulerMethod())

In [ ]:
n_ode.model.variables

In [ ]:
n_ode.pretrain(t_scale=1.0, step_size=0.2, n_epoch = 1)

In [ ]:
y0 = tf.constant([[3.0, 4.0]], dtype=tf.float64)
resid = lambda y: y*y-tf.constant([[2.0, 3.0]], dtype=tf.float64)
jac = lambda y: tf.eye(2, dtype=tf.float64)*y*2
neural_ode.ODESolvers.Newtons_method(y0, resid, jac)

In [ ]:
# Generate solution

In [7]:
#
N_n = int(2)
c = 0.1
k = 4.0
def oscilator(t, y):
    return np.array([y[1], -c*y[1]-k*y[0]])
t_final = 20.0
n_eval = int(501)
t_span = np.array([0.0, t_final])
y0 = np.array([1.0, 0.0])

In [8]:
sol = solve_ivp(oscilator, t_span, y0, t_eval=np.linspace(0, t_final, num=n_eval))

In [9]:
plt.plot(sol.t, sol.y[0,:])

In [10]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )

In [11]:
sol1 = n_ode.forward_solve(t_target[0:3], y_target[0,:])
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

dict_keys([])
dict_keys(['jac'])
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
tf.Tensor(
[[-0.99187975 -1.00750565]
 [-1.01579025 -1.0060636 ]], shape=(2, 2), dtype=float64)
tf.Tensor(
[[ 1.84454454e-04]
 [-1.54364415e-05]], shape=(2, 1), dtype=float64)
tf.Tensor([[ 1.00812025 -0.00750565]], shape=(1, 2), dtype=float64)
tf.Tensor(
[[-0.00788099]
 [ 0.00794184]], shape=(2, 1), dtype=float64)
tf.Tensor(
[[-0.99187975 -1.00750565]
 [-1.01579025

In [ ]:
# adjoint method

In [ ]:
loss, dl_dy, a = n_ode.adjoint_method(t_target[0:3], y_target[0:3,:])

In [ ]:
# fit

In [ ]:
n_epoch = 40
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_fold=5, adjoint_method=False)

In [ ]:
# Check derivatives

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss, dl_dp = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)
#loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target, x_external=f_ext_interp)
#dl_dp = a[2:]

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.0, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c+dp], [0.0, 1.0]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target, x_external=f_ext_interp)

In [ ]:
(loss2-loss)/dp

In [ ]:
dl_dp